coordinate2 implementation (build_batch)
==========

In [ ]:
"""
The coordinate module
=====================
This module handles coordinate systems and transformations in the phase space.
"""
from __future__ import print_function
import numpy as np
import sftrident.util as util
import sftrident.qft as qft

def transform_sphericalToLCC_single(oldCoord,m):
    """
    transforms spherical to lcc for a single onshell momentum

    -> move this to trafo
    """
    if len(oldCoord)!=3:
        raise ValueError("There are not exactly three coordinates! <%d> given."%(len(oldCoord)))
    E,cTh,phi = oldCoord
    rho = np.sqrt(E**2 - m**2)*(E>=1)
    sTh = np.sqrt(1-cTh**2)
    pm=(E-rho*cTh)/2.0
    return (pm,rho*sTh*np.cos(phi),rho*sTh*np.sin(phi))

def physArea_cond_single(pm,px,py,m,atol=1e-5,rtol=1e-4):
    """
    condition for momenta in light cone coordinates
    """
    posPM = (pm>atol)
    pmPos,pxPos,pyPos = [pm[posPM],px[posPM],py[posPM]]
    ppPos = (pxPos**2 + pyPos**2 + m**2)/(4.0*pmPos)
    massSQ = (ppPos+pmPos)**2 - pxPos**2 - pyPos**2 - (ppPos-pmPos)**2
    onshell = (np.abs(massSQ - m**2) < rtol * m**2)
    posPM[np.where(posPM)] = onshell
    return posPM


def transform_transverseToLCC_single(oldCoord,m):
    """
    transforms from transverse to lcc for a single onshell momentum
    """
    if len(oldCoord)!=3:
        raise ValueError("There are not exactly three coordinates! <%d> given."%(len(oldCoord)))
    y,pT,phi=oldCoord
    pm=0.5*np.exp(-y)*np.sqrt(pT**2 + m**2)
    return (pm,pT*np.cos(phi),pT*np.sin(phi))

def omegaCMS(ss,m):
    return (ss**2 - m**2)/(2.0*ss)

def energyCMS(ss,m):
    return (ss**2 + m**2)/(2.0*ss)

def cmsInitPara():
    return (energyCMS,omegaCMS)

def omegaLAB(ss,m):
    return (ss**2 - m**2)/(2.0*m)

def energyLAB(ss,m):
    return np.ones(ss.shape)*m

def labInitPara():
    return (energyLAB,omegaLAB)


#jacobians

def jac_sph2lcc(mom):
    pm = .5*(mom._0() - mom._3())
    mass_square = mom*mom
    rho = np.sqrt(mom._0()**2 - mass_square)
    return pm*rho

def jac_trans2lcc(mom):
    pm = .5*(mom._0() - mom._3())
    pT = np.sqrt(mom._1()**2 + mom._2()**2)
    return pm*pT


In [ ]:
class defaultClass(object):
    """
    default class for unmutual default class attributes
    """
    default = {}

    def __init__(self, **kwargs):
        [self.__setattr__(key,kwargs.pop(key,self.default[key]))for key in self.default.iterkeys()]
        if len(kwargs.keys())!=0:
            wrongargs = ",".join([key for key in kwargs.iterkeys()])
            raise AttributeError("%s has no keyword argument(s) named '%s'!"%(self.__class__,wrongargs))

class default_batch(defaultClass):
    default = {
            'coordSys':'lcc',
            'frame':'lab',
            'mass':1.0,
    }
    avialFrames = {
            'lab': labInitPara,
            'cms': cmsInitPara
    }
    avialCoordSys = {
                    'lcc':lambda x,m:x,
                    'sph':transform_sphericalToLCC_single,
                    'trans':transform_transverseToLCC_single
                    }
    def __init__(self,rawArrs,**kwargs):
        defaultClass.__init__(self,**kwargs) #update default values
        if not (self.coordSys in self.avialCoordSys.keys()):
            raise ValueError("There is no coordinate system called: <%s> !"%(self.coordSys))
        if not (self.frame in self.avialFrames.keys()):
            raise ValueError("There is no frame of reference called: <%s> !"%(self.frame))

        self.rawArrs = rawArrs #(7,N)
        self.dim, self.rawsize = self.rawArrs.shape
        self.__ssArr = self.rawArrs[0]
        
        self.__lccArr1, self.__lccArr2 = self.__transformLCCgrid()
        self.__buildE()
        self.__buildLccMesh3()
        self.__buildPhysArea()
        self.__buildKinGrids()
        
    def __transformLCCgrid(self):
        """
        transforms the inputed rawArrs to lccArrs
        """
        return [self.avialCoordSys[self.coordSys](arr,self.mass) for arr in util.groupList(self.rawArrs[1:],3)]
    
    def __buildE(self):
        self.__Earr = self.avialFrames[self.frame]()[0](self.__ssArr,self.mass)
        self.__rhoArr = np.sqrt(self.__Earr**2 - self.mass**2)
        
    def __buildLccMesh3(self):
        self.__lccArr3m = 0.5*(self.__Earr + self.__rhoArr) - self.__lccArr1[0]- self.__lccArr2[0]
        self.__lccArr3x = - self.__lccArr1[1] - self.__lccArr2[1]
        self.__lccArr3y = - self.__lccArr1[2] - self.__lccArr2[2]
        self.__lccArr3 = (self.__lccArr3m,self.__lccArr3x,self.__lccArr3y)
    
    def __buildPhysArea(self):
        #self.physArea = (self.__lccArr1[0]>0)&(self.__lccArr2[0]>0)&(self.__lccArr3[0]>0)
        #print(self.__lccArr1)
        self.physArea = physArea_cond_single(*self.__lccArr1,m=self.mass,atol=1e-3,rtol=1e-4)&physArea_cond_single(*self.__lccArr2,m=self.mass,atol=1e-3,rtol=1e-4)&physArea_cond_single(*self.__lccArr3,m=self.mass,atol=1e-3,rtol=1e-4)


    def __buildKinGrids(self):
        self.lccGrid1 = [el[self.physArea] for el in self.__lccArr1]
        #print('lcc1:',self.lccGrid1)
        self.lccGrid2 = [el[self.physArea] for el in self.__lccArr2]
        #print('lcc2:',self.lccGrid2)
        self.lccGrid3 = [el[self.physArea] for el in self.__lccArr3]
        #print('lcc3:',self.lccGrid3)
        self.ssGrid = self.__ssArr[self.physArea]
        self.EGrid = self.__Earr[self.physArea]
        self.size = len(self.EGrid)
        self.rhoGrid = self.__rhoArr[self.physArea]
        self.omegaGrid = self.avialFrames[self.frame]()[1](self.ssGrid,self.mass)
    
class build_batch(default_batch):
    def __init__(self,rawGrid,**kwargs):
        if len(rawGrid.shape)==1:
            default_batch.__init__(self,rawGrid[:,np.newaxis],**kwargs)
        elif len(rawGrid.shape)==2:
            default_batch.__init__(self,rawGrid.T,**kwargs)
        else:
            ValueError("Input of %s needs to be two dimensional! (<%s> given)"%(self.__class__,str(rawGrid.shape)))
            

Input test
======

In [ ]:
testInput1 = np.array([1,2,3,4,5,6,7])

testBatch1 = build_batch(testInput1,frame='cms')

print('rawsize',testBatch1.rawsize)
print('dim',testBatch1.dim)
print(testBatch1.rawArrs)

In [ ]:
import time

def buildArr_lcc_cart(Nss,N1m,N1x,N1y,N2m,N2x,N2y,eps=1e-4):
    ss = np.linspace(3.1,4.1,Nss)
    p1m = np.linspace(eps,1.5,N1m)
    p1x = np.linspace(-2,2,N1x)
    p1y = np.linspace(-2,2,N1y)
    p2m = np.linspace(eps,1.5,N2m)
    p2x = np.linspace(-2,2,N2x)
    p2y = np.linspace(-2,2,N2y)
    return util.cartesian_product(ss,p1m,p1x,p1y,p2m,p2x,p2y)

N = (2,3,4,5,6,7,8)

testInput2 = buildArr_lcc_cart(*N)

start = time.time()
testBatch2 = build_batch(testInput2)
end = time.time() - start

print('rawsize',testBatch2.rawsize)
print('dim',testBatch2.dim)
print('size',testBatch2.size)

for i,el in enumerate(testInput2):
    for ind in np.arange(testBatch2.dim):
        inputVal = el[ind]
        batchVal = testBatch2.rawArrs[ind,i]
        if inputVal!=batchVal:
            print("ERROR:")
            print('coord %d:'%ind)
            print('\tinput',)
            print('\tbatch',)
print('> inputtest done.')

In [ ]:
print("%1.2e"%end)

In [ ]:
import time
import sftrident.coordinate as coord

def buildArr_lcc_cart(Nss,N1m,N1x,N1y,N2m,N2x,N2y,eps=1e-4):
    ss = np.linspace(3.1,4.1,Nss)
    p1m = np.linspace(eps,1.5,N1m)
    p1x = np.linspace(-2,2,N1x)
    p1y = np.linspace(-2,2,N1y)
    p2m = np.linspace(eps,1.5,N2m)
    p2x = np.linspace(-2,2,N2x)
    p2y = np.linspace(-2,2,N2y)
    return util.cartesian_product(ss,p1m,p1x,p1y,p2m,p2x,p2y)

N = (2,3,4,5,6,7,8)

testInput2 = buildArr_lcc_cart(*N)

start = time.time()
testBatch3 = coord.coordinate(testInput2)
end = time.time() - start

print('rawsize',testBatch3.rawsize)
print('size',testBatch2.size)
print('dim',testBatch3.dim)

for i in np.arange(testBatch3.size):
    inputVal = testBatch2.lccGrid1[0][i]
    batchVal = testBatch3.lccGrid1[0][i]
    if inputVal!=batchVal:
        print("ERROR:")
        print('\tinput',inputVal)
        print('\tbatch',batchVal)
print('> inputtest done.')

In [ ]:
print("%1.2e"%end)

In [ ]:
print(testBatch2.lccGrid1[2].shape)

Benchmark (coord1 vs coord2)
=========

In [ ]:
import time
import sftrident.coordinate as coord

def buildArr_lcc_cart(Nss,N1m,N1x,N1y,N2m,N2x,N2y,eps=1e-4):
    ss = np.linspace(3.1,4.1,Nss)
    p1m = np.linspace(eps,1.5,N1m)
    p1x = np.linspace(-2,2,N1x)
    p1y = np.linspace(-2,2,N1y)
    p2m = np.linspace(eps,1.5,N2m)
    p2x = np.linspace(-2,2,N2x)
    p2y = np.linspace(-2,2,N2y)
    return util.cartesian_product(ss,p1m,p1x,p1y,p2m,p2x,p2y)

N = (2,30,4,5,30,7,8)

testInput = buildArr_lcc_cart(*N)

In [ ]:
start = time.time()
testBatchNEW = build_batch(testInput)
timeNEW = time.time() - start

start = time.time()
testBatchOLD = coord.coordinate(testInput)
timeOLD = time.time() - start


print('rawsizeOLD',testBatchOLD.rawsize)
print('sizeOLD',testBatchOLD.size)
print('dimOLD',testBatchOLD.dim)

print('rawsizeNEW',testBatchNEW.rawsize)
print('sizeOLD',testBatchNEW.size)
print('dimNEW',testBatchNEW.dim)

for i in np.arange(testBatch3.size):
    for el in np.arange(3):
        inputVal = testBatchOLD.lccGrid1[el][i]
        batchVal = testBatchNEW.lccGrid1[el][i]
        if inputVal!=batchVal:
            print("ERROR: (lcc1[%d])"%el)
            print('\tinput',inputVal)
            print('\tbatch',batchVal)
print('> lcc1 test done.')

for i in np.arange(testBatch3.size):
    for el in np.arange(3):
        inputVal = testBatchOLD.lccGrid2[el][i]
        batchVal = testBatchNEW.lccGrid2[el][i]
        if inputVal!=batchVal:
            print("ERROR: (lcc2[%d])"%el)
            print('\tinput',inputVal)
            print('\tbatch',batchVal)
print('> lcc2 test done.')

for i in np.arange(testBatch3.size):
    for el in np.arange(3):
        inputVal = testBatchOLD.lccGrid3[el][i]
        batchVal = testBatchNEW.lccGrid3[el][i]
        if inputVal!=batchVal:
            print("ERROR: (lcc3[%d])"%el)
            print('\tinput',inputVal)
            print('\tbatch',batchVal)
print('> lcc3 test done.')

In [ ]:
print('timeNEW',timeNEW)
print('timeOLD',timeOLD)